### LLM: post-train SFT + LoRA

In [1]:
%pip install transformers torch datasets trl tqdm accelerate peft

Note: you may need to restart the kernel to use updated packages.


In [19]:
# 1: 
# Импорты и зависимости

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from datasets import load_dataset
from tqdm.notebook import tqdm
import os

print("Импорты выполнены ✅")

# Проверка доступности GPU
print("Информация о системе:")
print(f"-->Доступен CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("-->Используется CPU")

Импорты выполнены ✅
Информация о системе:
-->Доступен CUDA: True
GPU: Tesla T4
Память GPU: 14.6 GB


In [3]:
# 2: 
# Загрузка и подготовка данных

def load_and_format_dataset():
    """Загрузка и форматирование датасета"""
    
    print("🔄 Загружаем датасет...")
    ds = load_dataset("d0rj/alpaca-cleaned-ru", split="train")
    
    # Функция для форматирования данных в диалоговый формат
    def format_alpaca_to_chat(example):
        """Преобразует пример Alpaca в диалоговый формат для Qwen"""
        if example['input']:
            user_message = f"{example['instruction']}\n\n{example['input']}"
        else:
            user_message = example['instruction']
        
        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": example['output']}
        ]
        
        # Форматируем в строку с использованием формата Qwen
        formatted_text = ""
        for msg in messages:
            if msg["role"] == "user":
                formatted_text += f"<|im_start|>user\n{msg['content']}<|im_end|>\n"
            else:
                formatted_text += f"<|im_start|>assistant\n{msg['content']}<|im_end|>\n"
        
        return {"text": formatted_text}
    
    # Применяем форматирование к датасету
    print("🔄 Форматируем данные...")
    formatted_ds = ds.map(format_alpaca_to_chat)
    
    # Проверяем форматирование на примере
    print("\nПример форматированного текста:")
    print(formatted_ds[11]['text'][:500] + "...")
    print(f"\n✅ Размер датасета: {len(formatted_ds)} примеров")
    
    return formatted_ds

# Загружаем данные
formatted_ds = load_and_format_dataset()

🔄 Загружаем датасет...
🔄 Форматируем данные...

Пример форматированного текста:
<|im_start|>user
Составьте список из десяти предметов, которые могут понадобиться человеку в походе.<|im_end|>
<|im_start|>assistant
Вот десять предметов, которые могут понадобиться человеку в походе:

1. Палатка - для укрытия и защиты от непогоды.
2. Спальный мешок - чтобы спать было тепло и комфортно
3. Переносная плита или гриль для костра – для приготовления пищи.
4. Охладитель со льдом или пакетами со льдом - чтобы скоропортящиеся продукты и напитки оставались холодными.
5. Фонарь или фонар...

✅ Размер датасета: 51760 примеров


In [4]:
# 3: 
# Тестирование модели ДО обучения

def test_before_training():
    """Тестирование модели до обучения"""
    print("Тестируем модель ДО обучения...")
    
    model_id = "Qwen/Qwen2.5-0.5B"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        dtype=torch.float16,
        device_map="auto"
    )
    
    questions_rus = [
        "сколько планет в нашей солнечной системе?",
        "расскажи стих",
        "когда собирать крыжовник?",
        "Как быстро выучить новый язык?"
    ]
    
    for i, question in enumerate(questions_rus, 1):
        print(f"\n{i}. Вопрос: {question}")
        
        messages = [{"role": "user", "content": question}]
        
        # Используем правильный метод для применения шаблона чата
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        inputs = tokenizer(text, return_tensors="pt").to(model.device)
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
        
        response = outputs[0][inputs['input_ids'].shape[1]:]
        answer = tokenizer.decode(response, skip_special_tokens=True)
        
        print(f"Ответ: {answer}")
        print("―" * 55)

# Запустить до обучения
test_before_training()

Тестируем модель ДО обучения...

1. Вопрос: сколько планет в нашей солнечной системе?
Ответ: сколько планет в нашей солнечной системе?玓
 Comey
шесть Comey
 Comey
шесть Comey
 Comey
шесть Comey
 Comey
шесть Comey

 Comey
 Comey
 Comey
 Comey
 Comey
 Comey
 Comey
 Comey
 Comey
 Comey
 Comey
 Comey
 Comey

―――――――――――――――――――――――――――――――――――――――――――――――――――――――

2. Вопрос: расскажи стих
Ответ: Привет! Это стихотворение из моего набора.
 Crimea
 Crimea, на дне Волги
 Настраивается на тиши и пустоты
 Особым племенем
 Осапа
 Осапа, на дне В
―――――――――――――――――――――――――――――――――――――――――――――――――――――――

3. Вопрос: когда собирать крыжовник?
Ответ: Отправь мне кисть и перечислите, какие части крыжовника вы хотите получить.

mite
когда собирать крыжовник?mite
 Comey
когда собирать крыжовник? Comey
 Comey
когда собирать крыжовник?
―――――――――――――――――――――――――――――――――――――――――――――――――――――――

4. Вопрос: Как быстро выучить новый язык?
Ответ: Чтобы быстро упорядочить знания, используйте методы запоминания. cre

In [7]:
# 4: 
# Настройка SFT с LoRA

from trl import SFTTrainer, SFTConfig
from peft import LoraConfig

def setup_training():
    """Настройка параметров обучения с SFTConfig"""
    
    model_id = "Qwen/Qwen2.5-0.5B"
    
    # Загрузка токенизатора и модели
    print(f"🔄 Загружаем модель {model_id}...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    
    # Определяем устройство (CPU/GPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Используем устройство: {device}")
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None
    )
    
    # Конфигурация LoRA
    peft_cfg = LoraConfig(
        r=8, 
        lora_alpha=16, 
        lora_dropout=0.05,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        task_type="CAUSAL_LM",
        bias="none"
    )
    
    # Создаем SFTConfig 
    cfg = SFTConfig(
        output_dir="./qwen2.5-0.5b-sft-ru",
        dataset_text_field="text",
        #max_seq_length=1024,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  
        logging_steps=50,
        num_train_epochs=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        warmup_ratio=0.03,
        report_to=None,
        run_name='qwen-lora-sft',
        save_steps=500,
        fp16=True,  # Включаем mixed precision для экономии памяти
        optim="adamw_torch",
        remove_unused_columns=False,
        packing=True,  # Экономит память
    )
    
    return model, tokenizer, peft_cfg, cfg

# Настраиваем обучение
model, tokenizer, peft_cfg, cfg = setup_training()

🔄 Загружаем модель Qwen/Qwen2.5-0.5B...
Используем устройство: cuda


In [12]:
# 5: Запуск SFT обучения с LoRA
def create_and_train_model(formatted_ds):
    """Создание и обучение модели с SFTConfig"""
    
    print("🔄 Создаем тренер...")
    
    trainer = SFTTrainer(
        model=model,
        processing_class=tokenizer,  # Используем processing_class
        args=cfg,
        train_dataset=formatted_ds,
        peft_config=peft_cfg,
        #dataset_text_field="text",  # Добавляем явное указание поля
        #max_seq_length=1024,  # Явно указываем max_seq_length
        #packing=True,  # Включаем packing для экономии памяти
    )
    
    # Запускаем обучение
    print("🚀 Начинаем обучение...")
    training_result = trainer.train()
    
    # Сохраняем модель
    print("💾 Сохраняем модель...")
    trainer.save_model()
    tokenizer.save_pretrained("./qwen2.5-0.5b-sft-ru")
    
    print("✅ Обучение завершено!")
    return trainer

# Запускаем обучение
trainer = create_and_train_model(formatted_ds)

Padding-free training is enabled, but the attention implementation is not set to a supported flash attention variant. Padding-free training flattens batches into a single sequence, and only the following implementations are known to reliably support this: flash_attention_2, flash_attention_3, kernels-community/flash-attn, kernels-community/flash-attn3, kernels-community/vllm-flash-attn3. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation` in the model configuration to one of these supported options or verify that your attention mechanism can handle flattened sequences.
You are using packing, but the attention implementation is not set to a supported flash attention variant. Packing gathers multiple samples into a single sequence, and only the following implementations are known to reliably support this: flash_attention_2, flash_attention_3, kernels-community/flash-attn, kernels-community/flash-attn3, kernels-community/vllm-fla

🔄 Создаем тренер...


Packing train dataset: 100%|██████████| 51760/51760 [00:00<00:00, 64309.12 examples/s]
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


🚀 Начинаем обучение...


Step,Training Loss
50,1.915500
100,1.672900
150,1.636400
200,1.643400
250,1.613700
300,1.603200
350,1.627900
400,1.608200
450,1.606100
500,1.596200


💾 Сохраняем модель...
✅ Обучение завершено!


In [17]:
%pip install --upgrade tqdm ipywidgets jupyter

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached notebook-7.5.0-py3-none-any.whl.metadata (10 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached jupyterlab-4.5.0-py3-none-any.whl.metadata (16 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached jupyter_lsp-2.3.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.17.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached jupyterlab_server-2.28.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached notebook_shim-0.2.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached tomli-2.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached argon2_cffi-25.1.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached jupyter_events-0.12.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached jupyter_server_terminals-0.5.3-py3-none-any.whl.metadata (5.6 kB)
  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached prometheus_client-0.23

In [18]:
#  6: (второй вариант)
# Тестирование модели ПОСЛЕ обучения

def test_after_training():
    """Тестирование модели после обучения"""
    
    print("\nТестируем модель ПОСЛЕ обучения...")
    
    # Загружаем обученную модель
    model_path = "./qwen2.5-0.5b-sft-ru"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )
    
    questions_rus = [
        "сколько планет в нашей солнечной системе?",
        "расскажи стих",
        "когда собирать крыжовник?",
        "Как быстро выучить новый язык?"
    ]
    
    print("Ответы обученной модели:\n")
    
    for i, question in enumerate(questions_rus, 1):
        print(f"\n{i}. Вопрос: {question}")
        
        # Форматируем сообщение - добавляем system промпт как при обучении
        messages = [
            {"role": "system", "content": "Ты - полезный AI ассистент."},
            {"role": "user", "content": question}
        ]
        
        # Применяем шаблон чата
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        inputs = tokenizer(text, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        # Генерация
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
        
        # Извлекаем ответ
        response = outputs[0][inputs['input_ids'].shape[1]:]
        answer = tokenizer.decode(response, skip_special_tokens=True)
        
        print(f"Ответ: {answer.strip()}")
        print("―" * 55)

# Тестируем после обучения
test_after_training()


Тестируем модель ПОСЛЕ обучения...
Ответы обученной модели:


1. Вопрос: сколько планет в нашей солнечной системе?
Ответ: Солнечная система состоит из 8 солнечных行星, которые являются солнцем и оставляют свет. Они называются: Марс, Венера, Юпитер, Сатурн, Галактика, Земля, Марс и Солнце. Солнце, как известно, находится в центре нашей солнечной системы, а остальные солнечные行星 находятся в разных местах от него. Кроме того, солнечные системы также включают теневой и нененефетские солнечные системы, которые находятся в разные части нашей солнечной системы. Однако, согласно оценкам, в нашей солнечной системе оставляются около 1700 нененефетских солнечных систем. Некоторые из них являются менее развитыми, а некоторые более развитыми, чем в нашей системе. Солнечная система состоит из 8 солнечных планет, включая Марс, Венеру, Юпитер, Сатурн, Галактику, Землю, Мар
―――――――――――――――――――――――――――――――――――――――――――――――――――――――

2. Вопрос: расскажи стих
Ответ: Одна утка, накатавшаяся на землю, пыталась

In [ ]:
# 7: Очистка памяти
def cleanup_memory():
    """Очистка памяти GPU"""
    import gc
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    print("✅ Память очищена")

# Запустить при необходимости
cleanup_memory()